In [8]:
from pydicom import dcmread
#from pydicom.data import get_testdata_file

In [9]:
file = open("Data\CTHead2.dcm", "rb") #open dicom file
file.seek(-8192,2)
file.seek(0)
byte = file.read(1410) #read bytes corresponding to header
f = open("Data\dummy_header.dcm", "wb") #open a file to dump the read bytes
f.write(byte) # write the read bytes
f.close()
#while byte:

In [10]:
from PIL import Image
import os
import pydicom
from pydicom.uid import ExplicitVRLittleEndian
import pydicom._storage_sopclass_uids
import numpy as np
from matplotlib import image
from matplotlib import pyplot
import time


In [11]:
def SetCommonFields(ds):
    meta = ds.file_meta
    meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian  
    ds.is_little_endian = True
    ds.is_implicit_VR = False
    ds.Modality = "CT"
    ds.BitsStored = 8
    ds.BitsAllocated = 8
    ds.SamplesPerPixel = 1
    ds.HighBit = 7
    ds.ImagesInAcquisition = "1"
    ds.InstanceNumber = 1
    ds.ImagePositionPatient = [0.000, 0.000, 0.000]#r"0\0\1"
    ds.RescaleIntercept = "0"
    ds.RescaleSlope = "1"
    ds.PixelSpacing = r"3.2\3.2"
    ds.PhotometricInterpretation = "MONOCHROME2"
    ds.InstanceNumber = 0
    ds.SliceThickness = 2.0

In [12]:
fpath = "Data\CTHead2.dcm"
ds = dcmread(fpath)
directory = "Data\Input"
if not os.path.isdir('Data\EightBitOut'):
    os.mkdir('Data\EightBitOut')
save_path = 'Data\EightBitOut'
if not os.path.isdir('Data\DcmOut'):
    os.mkdir('Data\DcmOut')
out_dicom_path = 'Data\DcmOut'
num = 1
pos = 0.00
for filename in os.listdir(directory):
    in_name = os.path.join(directory, filename)
    out_name = os.path.join(save_path, filename)
    img = Image.open(in_name)
    newimg = img.convert(mode='L', colors=8)
    newimg.save(out_name)
SetCommonFields(ds)
time.sleep(1)
for filename in os.listdir(save_path):
    image_path = os.path.join(save_path, filename)
    image_arr = image.imread(image_path)
    image2d = image_arr.astype(np.uint8)   
    ds.InstanceNumber += num
    ds.ImagePositionPatient = [0, pos, pos]
    ds.PixelData = image2d.tobytes()
    ds.Rows = image2d.shape[0]
    ds.Columns = image2d.shape[1]
    name_dcm = str(os.path.join(out_dicom_path, filename))+".dcm"
    ds.save_as(name_dcm)
    pos += 2.00